In [1]:
# Import necessary libraries
import pandas as pd
from dotenv import load_dotenv
from query_tool import DatabaseQueryTool
from utils import read_sql_file
import os

# Load environment variables
load_dotenv('config/.env')

# Load configurations from environment variables
snowflake_config = {
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

bigquery_config = {
    "project": os.getenv("BIGQUERY_PROJECT_ID"),
    "credentials": os.getenv("BIGQUERY_CREDENTIALS_FILE"),
}

In [2]:
# Initialize the query tool
query_tool = DatabaseQueryTool(snowflake_config, bigquery_config)

In [3]:
# Read SQL queries from files
snowflake_query = read_sql_file("queries/snowflake_query.sql")
bigquery_query = read_sql_file("queries/bigquery_query.sql")

In [4]:
# Execute query on BigQuery
print("Running BigQuery query...")
bigquery_df = query_tool.query_bigquery(bigquery_query)
print(f"BigQuery query completed. Retrieved {len(bigquery_df)} rows.")


INFO:root:Executing query on BigQuery...
INFO:root:Initializing BigQuery client...


Running BigQuery query...


INFO:root:BigQuery: Rows=704187, Columns=10, Data=0.00 MB, Time=1.48 seconds.


BigQuery query completed. Retrieved 704187 rows.


In [5]:
# Execute query on Snowflake
print("Running Snowflake query...")
snowflake_df = query_tool.query_snowflake(snowflake_query)
print(f"Snowflake query completed. Retrieved {len(snowflake_df)} rows.")

INFO:root:Executing query on Snowflake...
INFO:root:Initializing Snowflake connection...
INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.12.3, Python Version: 3.12.4, Platform: macOS-14.3-arm64-arm-64bit
INFO:snowflake.connector.connection:Connecting to GLOBAL Snowflake domain
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


Running Snowflake query...


INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:root:Snowflake Query Completed: Rows=14481, Columns=9, Processing Time=1.43 seconds.


Snowflake query completed. Retrieved 14481 rows.


In [6]:
# Inner Join Results
print("Performing left join on orders, users, and items as skus...")
join_columns = ["order_id", "user_id", "item_sku"]
output_columns = [
    "event_timestamp_utc", "order_id", "user_id", "event_action", "item_sku", "item_price", "traffic_source",
    "user_country", "device_category"
]

result_df = query_tool.join_results(bigquery_df, snowflake_df, join_columns, output_columns)

INFO:root:Joining results...
INFO:root:Initial df1 shape: (704187, 10), df2 shape: (14481, 9)


Performing left join on orders, users, and items as skus...


INFO:root:Shape of joined DataFrame: (39267, 16)
INFO:root:Shape of final output DataFrame: (39267, 9)


In [7]:
# Output results
print("Left join completed. Here are the first few rows of the result:")
print(result_df.head())

Left join completed. Here are the first few rows of the result:
        event_timestamp_utc    order_id              user_id     event_action  \
0 2016-11-23 18:23:30+00:00  1479925410  6298528466956890303  Quickview Click   
1 2016-11-23 18:23:30+00:00  1479925410  6298528466956890303  Quickview Click   
2 2016-11-23 18:23:30+00:00  1479925410  6298528466956890303  Quickview Click   
3 2016-11-23 18:23:30+00:00  1479925410  6298528466956890303  Quickview Click   
4 2016-11-23 18:23:30+00:00  1479925410  6298528466956890303  Quickview Click   

       item_sku item_price    traffic_source   user_country device_category  
0  GGOEGAAX0231    3990000  sites.google.com  United States         desktop  
1  GGOEGAAX0231    4990000  sites.google.com  United States         desktop  
2  GGOEGAAX0231    4990000  sites.google.com  United States         desktop  
3  GGOEGAAX0231    4990000  sites.google.com  United States         desktop  
4  GGOEGAAX0098    1200000  sites.google.com  United States

In [8]:
# Add a new column to the result - if the event_action is "Add to Cart", then the new column "is_purchase" should be 1, otherwise 0.
result_df["is_purchase"] = result_df["event_action"].apply(lambda x: 1 if x == "Add to Cart" else 0)

In [9]:
# Write the result_df to a Snowflake table named "orders_items_events"
table_name = '"orders_items_events"'
query_tool.write_to_snowflake(result_df, table_name)

INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:root:Successfully inserted 39267 rows into Snowflake table '"orders_items_events"'.


,event_timestamp_utc,order_id,user_id,event_action,item_sku,item_price,traffic_source,user_country,device_category,is_purchase
0,2016-11-23 18:23:30,1479925410,6298528466956890303,Quickview Click,GGOEGAAX0231,3990000,sites.google.com,United States,desktop,0
1,2016-11-23 18:23:30,1479925410,6298528466956890303,Quickview Click,GGOEGAAX0231,4990000,sites.google.com,United States,desktop,0
2,2016-11-23 18:23:30,1479925410,6298528466956890303,Quickview Click,GGOEGAAX0231,4990000,sites.google.com,United States,desktop,0
3,2016-11-23 18:23:30,1479925410,6298528466956890303,Quickview Click,GGOEGAAX0231,4990000,sites.google.com,United States,desktop,0
4,2016-11-23 18:23:30,1479925410,6298528466956890303,Quickview Click,GGOEGAAX0098,1200000,sites.google.com,United States,desktop,0
...,...,...,...,...,...,...,...,...,...,...
39262,2016-08-05 09:44:23,1470390263,7467888750938090707,Add to Cart,GGOEGBRJ037299,99990000,analytics.google.com,Germany,desktop,1
39263,2016-08-05 09:44:23,1470390263,7467888750938090707,Add to Cart,GGOEGBRJ037299,79990000,analytics.google.com,Germany,desktop,1
39264,2016-08-05 09:44:23,1470390263,7467888750938090707,Add to Cart,GGOEGBRJ037299,99990000,analytics.google.com,Germany,desktop,1
39265,2016-08-05 09:44:23,1470390263,7467888750938090707,Add to Cart,GGOEGBRJ037299,79990000,analytics.google.com,Germany,desktop,1


In [10]:
## Import logs
import_logs = query_tool.import_logs
import_logs


,source,query,row_count,col_count,data_mb,time_sec,timestamp
0,BigQuery,"SELECT \n event_timestamp_utc,\n event_actio...",704187,10,0.0,1.482298,2024-11-25 12:33:04
1,Snowflake,"SELECT \n ""order_date"", \n ""order_id"", \...",14481,9,NaN,1.427480,2024-11-25 12:33:06


In [11]:
## Join logs
join_logs = query_tool.join_logs
join_logs

,df1_shape,df2_shape,join_columns,output_columns,result_shape,duplicate_rows,timestamp,duplicated_rows
0,"(704187, 10)","(14481, 9)","[order_id, user_id, item_sku]","[event_timestamp_utc, order_id, user_id, event...","(39267, 16)",NaN,2024-11-25 12:33:06,9136.0
